In [9]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('./Dataset/phl_exoplanet_catalog_2019.csv')

In [11]:
df.head()

,P_NAME,P_STATUS,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_UPDATED,...,P_HABZONE_CON,P_TYPE_TEMP,P_HABITABLE,P_ESI,S_CONSTELLATION,S_CONSTELLATION_ABR,S_CONSTELLATION_ENG,P_RADIUS_EST,P_MASS_EST,P_SEMI_MAJOR_AXIS_EST
0,11 Com b,3.0,6165.86330,-476.74200,476.74200,NaN,NaN,NaN,2007,2014-05-14,...,0,Hot,0,0.083813,Coma Berenices,Com,Berenice's Hair,12.082709,6165.86330,1.29
1,11 UMi b,3.0,4684.78480,-794.57001,794.57001,NaN,NaN,NaN,2009,2018-09-06,...,0,Hot,0,0.082414,Ursa Minor,UMi,Little Bear,12.229641,4684.78480,1.53
2,14 And b,3.0,1525.57440,NaN,NaN,NaN,NaN,NaN,2008,2014-05-14,...,0,Hot,0,0.081917,Andromeda,And,Andromeda,12.848516,1525.57440,0.83
3,14 Her b,3.0,1481.07850,-47.67420,47.67420,NaN,NaN,NaN,2002,2018-09-06,...,0,Cold,0,0.145241,Hercules,Her,Hercules,12.865261,1481.07850,2.93
4,16 Cyg B b,3.0,565.73385,-25.42624,25.42624,NaN,NaN,NaN,1996,2018-09-06,...,1,Warm,0,0.368627,Cygnus,Cyg,Swan,13.421749,565.73385,1.66


In [12]:
temp = df['P_HABITABLE']
temp.value_counts()

P_HABITABLE
0    3993
2      34
1      21
Name: count, dtype: int64

In [13]:
# Missing value percentage
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing_pct[missing_pct > 0]

P_DETECTION_MASS    100.000000
P_GEO_ALBEDO        100.000000
S_MAGNETIC_FIELD    100.000000
S_DISC              100.000000
P_ATMOSPHERE        100.000000
                       ...    
P_APASTRON            1.729249
P_PERIASTRON          1.729249
P_DISTANCE            1.729249
P_TYPE                0.419960
S_DISTANCE            0.148221
Length: 92, dtype: float64

In [20]:
for col in df.columns:
    print(col)

P_NAME
P_STATUS
P_MASS
P_MASS_ERROR_MIN
P_MASS_ERROR_MAX
P_RADIUS
P_RADIUS_ERROR_MIN
P_RADIUS_ERROR_MAX
P_YEAR
P_UPDATED
P_PERIOD
P_PERIOD_ERROR_MIN
P_PERIOD_ERROR_MAX
P_SEMI_MAJOR_AXIS
P_SEMI_MAJOR_AXIS_ERROR_MIN
P_SEMI_MAJOR_AXIS_ERROR_MAX
P_ECCENTRICITY
P_ECCENTRICITY_ERROR_MIN
P_ECCENTRICITY_ERROR_MAX
P_INCLINATION
P_INCLINATION_ERROR_MIN
P_INCLINATION_ERROR_MAX
P_OMEGA
P_OMEGA_ERROR_MIN
P_OMEGA_ERROR_MAX
P_TPERI
P_TPERI_ERROR_MIN
P_TPERI_ERROR_MAX
P_ANGULAR_DISTANCE
P_IMPACT_PARAMETER
P_IMPACT_PARAMETER_ERROR_MIN
P_IMPACT_PARAMETER_ERROR_MAX
P_TEMP_MEASURED
P_GEO_ALBEDO
P_GEO_ALBEDO_ERROR_MIN
P_GEO_ALBEDO_ERROR_MAX
P_DETECTION
P_DETECTION_MASS
P_DETECTION_RADIUS
P_ALT_NAMES
P_ATMOSPHERE
S_NAME
S_RA
S_DEC
S_MAG
S_DISTANCE
S_DISTANCE_ERROR_MIN
S_DISTANCE_ERROR_MAX
S_METALLICITY
S_METALLICITY_ERROR_MIN
S_METALLICITY_ERROR_MAX
S_MASS
S_MASS_ERROR_MIN
S_MASS_ERROR_MAX
S_RADIUS
S_RADIUS_ERROR_MIN
S_RADIUS_ERROR_MAX
S_TYPE
S_AGE
S_AGE_ERROR_MIN
S_AGE_ERROR_MAX
S_TEMPERATURE
S_TEMPERATURE

In [21]:
keep_cols = [
    'P_MASS',                 # 1
    'P_RADIUS',               # 2
    'P_PERIOD',               # 3
    'P_DENSITY',              # 4
    'P_GRAVITY',              # 5
    'P_ESCAPE',               # 6
    'P_SEMI_MAJOR_AXIS',      # 7
    'P_ECCENTRICITY',         # 8
    'P_DISTANCE',             # 9
    'P_DISTANCE_EFF',         # 10
    'P_FLUX',                 # 11
    'P_TEMP_EQUIL',           # 12
    'S_MASS',                 # 13
    'S_RADIUS',               # 14
    'S_TEMPERATURE',          # 15
    'S_LUMINOSITY',           # 16
    'S_METALLICITY',          # 17
    'S_AGE',                  # 18
    'P_TYPE',                 # 19
    'P_TYPE_TEMP',            # 20
    'S_TYPE',                 # 21
    'S_TYPE_TEMP',            # 22
    'P_HABITABLE'             # 23 (target)
]
df_model = df[keep_cols]

In [23]:
missing_count = df_model.isna().sum().sort_values(ascending=False)
missing_count[missing_count > 0]

P_DENSITY            3342
P_GRAVITY            3342
P_ESCAPE             3342
S_TYPE               2678
P_ECCENTRICITY       2668
P_MASS               2450
S_AGE                2017
P_SEMI_MAJOR_AXIS    1681
S_METALLICITY        1206
P_RADIUS              909
S_MASS                765
P_TYPE_TEMP           327
P_TEMP_EQUIL          327
P_FLUX                327
S_RADIUS              325
S_LUMINOSITY          262
S_TEMPERATURE         207
S_TYPE_TEMP           136
P_PERIOD              110
P_DISTANCE_EFF         70
P_DISTANCE             70
P_TYPE                 17
dtype: int64

In [24]:
df_model = df[keep_cols]

cat_cols = df_model.select_dtypes(include=['object']).columns
num_cols = df_model.select_dtypes(include=['int64', 'float64']).columns

# remove target from numerical list
num_cols = num_cols.drop('P_HABITABLE')


In [25]:
for col in num_cols:
    df_model[col] = df_model[col].fillna(df_model[col].median())


C:\Users\laeeq\AppData\Local\Temp\ipykernel_57392\1496585621.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[col] = df_model[col].fillna(df_model[col].median())


In [26]:
for col in cat_cols:
    df_model[col] = df_model[col].fillna(df_model[col].mode()[0])


C:\Users\laeeq\AppData\Local\Temp\ipykernel_57392\741331730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[col] = df_model[col].fillna(df_model[col].mode()[0])


In [34]:
# Verify no missing values remain
missing_count_after = df_model.isna().sum()
missing_count_after[missing_count_after > 0]

Series([], dtype: int64)